In [4]:
import sys
from contextlib import redirect_stdout
!pip install tensorflow
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, concatenate, Lambda
from tensorflow.keras.backend import clear_session

from os import listdir
from xml.dom.minidom import parse
from nltk.tokenize import word_tokenize

import string
import re
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Dataset

In [5]:
class Dataset:
    """
    Reads all XML files in a given directory and generates a list of sentences.
    Each sentence is a list of tuples (word, start, end, tag)
    """
    def __init__(self, datadir):
        self.data = {}
        # Process each file in the directory
        for f in listdir(datadir):

            # Parse the XML file, get a DOM tree
            tree = parse(datadir + "/" + f)

            # Process each sentence in the file
            sentences = tree.getElementsByTagName("sentence")
            for s in sentences:
                sid = s.attributes["id"].value  # Get the sentence ID
                stext = s.attributes["text"].value  # Get the sentence text
                entities = s.getElementsByTagName("entity")

                spans = []
                for e in entities:
                    # For discontinuous entities, we only get the first span.
                    # Obviously, the model will not work for this type of entities. Their encoding is complex.
                    # But it doesn't matter much, there are very few.
                    (start, end) = e.attributes["charOffset"].value.split(";")[0].split("-")
                    typ = e.attributes["type"].value
                    spans.append((int(start), int(end), typ))

                # Convert the sentence into a sequence of tokens
                tokens = self.__tokenize(stext)

                # Add the Gold Standard tag
                self.data[sid] = []
                for i in range(0, len(tokens)):
                    # Check if the token is part of the tag
                    tokens[i]['tag'] = self.__get_tag(tokens[i], spans)
                    self.data[sid].append(tokens[i])

    ## --------- Tokenize the sentence -----------
    ## Tokenizes the sentence
    def __tokenize(self, txt):
        offset = 0
        tks = []
        ## word_tokenize separates the words
        for t in word_tokenize(txt):
            ## Manage the position (using the offset) where each token should appear
            offset = txt.find(t, offset)
            tks.append({'lc_form': t.lower(), 'form': t, 'start': offset, 'end': offset + len(t) - 1})
            offset += len(t)

        ## List of tuples
        return tks

    ## --------- Get the Tag -----------
    def __get_tag(self, token, spans):
        for (spanS, spanE, spanT) in spans:
            if token['start'] == spanS and token['end'] <= spanE:
                return "B-" + spanT
            elif token['start'] >= spanS and token['end'] <= spanE:
                return "I-" + spanT
        return "O"

    ## Iterator over the sentences in the dataset
    def sentences(self):
        for sid in self.data:
            yield self.data[sid]

    ## Iterator over the sentence IDs
    def sentence_ids(self):
        for sid in self.data:
            yield sid

    ## Get a sentence by its ID
    def get_sentence(self, sid):
        return self.data[sid]

    ## Get sentences as lists
    def tokens(self):
        for sid in self.data:
            s = []
            for w in self.data[sid]:
                s.append((sid, w['form'], w['start'], w['end']))
            yield s

# Codemaps

In [6]:
class Codemaps:

    def __init__(self, data, maxlen=None, suflen=None):
        """
        Default constructor
        :param data:
        :param maxlen:
        :param suflen:
        """
        if isinstance(data, Dataset) and maxlen is not None and suflen is not None:
            self.__create_indexs(data, maxlen, suflen)

        elif type(data) == str and maxlen is None and suflen is None:
            self.__load(data)

        else:
            print('codemaps: Invalid or missing parameters in constructor')
            exit()

    def __create_indexs(self, data, maxlen, suflen):
        """
        Create indices from the training corpus
        Extracts all words and labels in the given sentences and creates indices to encode them as numbers
        :param data:
        :param maxlen:
        :param suflen:
        :return:
        """

        self.maxlen = maxlen
        self.suflen = suflen
        words = set([])
        lc_words = set([])
        sufs = set([])
        labels = set([])

        for s in data.sentences():
            for t in s:
                words.add(t['form'])
                sufs.add(t['lc_form'][-self.suflen:])
                labels.add(t['tag'])

        self.word_index = {w: i + 2 for i, w in enumerate(list(words))}
        self.word_index['PAD'] = 0  # Padding
        self.word_index['UNK'] = 1  # Unknown words

        self.suf_index = {s: i + 2 for i, s in enumerate(list(sufs))}
        self.suf_index['PAD'] = 0  # Padding
        self.suf_index['UNK'] = 1  # Unknown suffixes

        self.label_index = {t: i + 1 for i, t in enumerate(list(labels))}
        self.label_index['PAD'] = 0  # Padding

    def __load(self, name):
        """
        Load the indices
        :param name:
        :return:
        """
        self.maxlen = 0
        self.suflen = 0
        self.word_index = {}
        self.suf_index = {}
        self.label_index = {}

        with open(name + ".idx") as f:
            for line in f.readlines():
                (t, k, i) = line.split()
                if t == 'MAXLEN':
                    self.maxlen = int(k)
                elif t == 'SUFLEN':
                    self.suflen = int(k)
                elif t == 'WORD':
                    self.word_index[k] = int(i)
                elif t == 'SUF':
                    self.suf_index[k] = int(i)
                elif t == 'LABEL':
                    self.label_index[k] = int(i)

    def save(self, name):
        """
        Save the indices
        :param name:
        :return:
        """
        with open(name + ".idx", "w") as f:
            print('MAXLEN', self.maxlen, "-", file=f)
            print('SUFLEN', self.suflen, "-", file=f)
            for key in self.label_index: print('LABEL', key, self.label_index[key], file=f)
            for key in self.word_index: print('WORD', key, self.word_index[key], file=f)
            for key in self.suf_index: print('SUF', key, self.suf_index[key], file=f)

    def encode_words(self, data):
        """
        Encodes X (text to indices)
        :param data:
        :return:
        """
        # Encode the words and add padding
        Xw = [[self.word_index[w['form']] if w['form'] in self.word_index else self.word_index['UNK'] for w in s] for s
              in data.sentences()]
        Xw = pad_sequences(maxlen=self.maxlen, sequences=Xw, padding="post", value=self.word_index['PAD'])
        # Encode the suffixes and add padding
        Xs = [[self.suf_index[w['lc_form'][-self.suflen:]] if w['lc_form'][-self.suflen:] in self.suf_index else
               self.suf_index['UNK'] for w in s] for s in data.sentences()]
        Xs = pad_sequences(maxlen=self.maxlen, sequences=Xs, padding="post", value=self.suf_index['PAD'])
        # Return the sentences
        return [Xw, Xs]

    def encode_labels(self, data):
        """
        Encodes labels and adds padding
        :param data:
        :return:
        """
        Y = [[self.label_index[w['tag']] for w in s] for s in data.sentences()]
        Y = pad_sequences(maxlen=self.maxlen, sequences=Y, padding="post", value=self.label_index["PAD"])
        return np.array(Y)

    def get_n_words(self):
        """
        Returns the number of word indices
        :return:
        """
        return len(self.word_index)

    def get_n_sufs(self):
        """
        Returns the number of suffix indices
        :return:
        """
        return len(self.suf_index)

    def get_n_labels(self):
        """
        Returns the number of labels
        :return:
        """
        return len(self.label_index)

    def word2idx(self, w):
        """
        Returns the index of a word
        :param w:
        :return:
        """
        return self.word_index[w]

    def suff2idx(self, s):
        """
        Returns the index of a suffix
        :param s:
        :return:
        """
        return self.suf_index[s]

    def label2idx(self, l):
        """
        Returns the index of a label
        :param l:
        :return:
        """
        return self.label_index[l]

    def idx2label(self, i):
        """
        Returns the label given an index
        :param i:
        :return:
        """
        for l in self.label_index:
            if self.label_index[l] == i:
                return l
        raise KeyError

# Train

In [7]:
def build_network(codes):
    """
    Function that builds the neural network.
    :param codes:
    :return:
    """
    global lstm_units, embeddingW_dim, embeddingS_dim
    
    # sizes
    n_words = codes.get_n_words() # Number of words in the vocabulary
    n_sufs = codes.get_n_sufs() # Number of suffixes in the vocabulary
    n_labels = codes.get_n_labels() # Number of word classes
    max_len = codes.maxlen # Maximum length of each sentence, for padding

    inptW = Input(shape=(max_len,))  # Word embedding layer
    embW = Embedding(input_dim=n_words, output_dim=embeddingW_dim,
                     input_length=max_len, mask_zero=True)(inptW)

    inptS = Input(shape=(max_len,))  # Suffix embedding layer
    embS = Embedding(input_dim=n_sufs, output_dim=embeddingS_dim,
                     input_length=max_len, mask_zero=True)(inptS)

    dropW = Dropout(0.1)(embW)
    dropS = Dropout(0.1)(embS)
    drops = concatenate([dropW, dropS])

    # biLSTM
    bilstm = Bidirectional(LSTM(units=lstm_units, return_sequences=True,
                                recurrent_dropout=0.1))(drops)
    # Fully connected layer with softmax activation function to obtain the labels
    out = TimeDistributed(Dense(n_labels, activation="softmax"))(bilstm)

    # Build and compile the model
    model = Model([inptW, inptS], out)
    model.compile(optimizer="adam",
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])

    return model

# Main

In this case, we comment the code line by line and use it as a base case to analyze the effect of modifying different parameters. The Dataset function assigns a series of features to each word: the word in lowercase, the raw word, the position of the first and last character in the sentence, and the tag according to B-I-O schema.

Then, a vocabulary is created, and each word and suffix is assigned the corresponding index. This way, the dataset is transformed into vectors, which are then used to train the model and make predictions.

In [8]:
# Directory with the files
traindir = 'data/train'
validationdir = 'data/devel'
modelname = 'models/model_1'

# Load the training and validation corpora
# ******************************************************************
# The following features are assigned to each word:
# - lc_form: word in lowercase
# - form: raw word
# - start: position of the first character in the sentence
# - end: position of the last character in the sentence
# - tag: word classification, according to B-I-O
# ******************************************************************
traindata = Dataset(traindir)
valdata = Dataset(validationdir)

# Create the indices
# ******************************************************************
# When initializing the object of the Codemaps class, sets with index identifiers are created for each word, suffix, and class.
# It is used to encode sentences
# ******************************************************************
max_len = 150 # Maximum length for padding
suf_len = 5 # Last characters of each word to obtain suffix
codes = Codemaps(traindata, max_len, suf_len)

# Build the network
# ******************************************************************
# Bidirectional LSTM model, with Embedding and Dropout layer
# ******************************************************************
# These are some model parameters (they are imported in the build_network function as global variables)
lstm_units = 10
embeddingW_dim = 50
embeddingS_dim = 50

model = build_network(codes)
model.summary()

# Map the datasets
# ******************************************************************
# Encodes each sentence by assigning the corresponding index to each word or suffix, and applies padding
# Also encodes the classifications
# ******************************************************************
Xt = codes.encode_words(traindata)
Yt = codes.encode_labels(traindata)
Xv = codes.encode_words(valdata)
Yv = codes.encode_labels(valdata)

# Train the model
model.fit(Xt, Yt, batch_size=32, epochs=10, validation_data=(Xv, Yv), verbose=1)

# Save the model and the indices
model.save(modelname)
codes.save(modelname)



Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 150)]                0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 150)]                0         []                            
                                                                                                  
 embedding (Embedding)       (None, 150, 50)              483800    ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, 150, 50)              247850    ['input_2[0][0]']             
                                                                                            

INFO:tensorflow:Assets written to: models/model_1\assets


# Change dimensions for Embbeding layer

Results do not improve, lower dimensions seem to work better.

In [9]:
dimensions = [10, 300]
model_names = ['embedding_10_dim', 'embedding_300_dim']

for dimension, model_name in zip(dimensions, model_names):
    clear_session()
    
    embeddingW_dim, embeddingS_dim = dimension, dimension
    model = build_network(codes)
    
    model.fit(Xt, Yt, batch_size=32, epochs=10, validation_data=(Xv, Yv), verbose=1)
    model.save(model_name)

Epoch 1/10
170/170 [==============================] - 43s 214ms/step - loss: 1.0012 - accuracy: 0.8736 - val_loss: 0.5454 - val_accuracy: 0.8781
Epoch 2/10
170/170 [==============================] - 37s 219ms/step - loss: 0.4557 - accuracy: 0.8834 - val_loss: 0.4087 - val_accuracy: 0.8790
Epoch 3/10
170/170 [==============================] - 37s 216ms/step - loss: 0.3141 - accuracy: 0.8941 - val_loss: 0.3073 - val_accuracy: 0.9104
Epoch 4/10
170/170 [==============================] - 36s 210ms/step - loss: 0.2402 - accuracy: 0.9312 - val_loss: 0.2662 - val_accuracy: 0.9275
Epoch 5/10
170/170 [==============================] - 33s 194ms/step - loss: 0.2075 - accuracy: 0.9374 - val_loss: 0.2453 - val_accuracy: 0.9297
Epoch 6/10
170/170 [==============================] - 35s 207ms/step - loss: 0.1832 - accuracy: 0.9403 - val_loss: 0.2208 - val_accuracy: 0.9352
Epoch 7/10
170/170 [==============================] - 38s 225ms/step - loss: 0.1520 - accuracy: 0.9555 - val_loss: 0.1949 - val_ac

INFO:tensorflow:Assets written to: embedding_10_dim\assets


Epoch 1/10
170/170 [==============================] - 59s 305ms/step - loss: 0.6016 - accuracy: 0.8879 - val_loss: 0.2912 - val_accuracy: 0.9237
Epoch 2/10
170/170 [==============================] - 56s 329ms/step - loss: 0.2067 - accuracy: 0.9426 - val_loss: 0.2105 - val_accuracy: 0.9457
Epoch 3/10
170/170 [==============================] - 53s 311ms/step - loss: 0.1361 - accuracy: 0.9620 - val_loss: 0.1710 - val_accuracy: 0.9582
Epoch 4/10
170/170 [==============================] - 53s 314ms/step - loss: 0.0904 - accuracy: 0.9769 - val_loss: 0.1572 - val_accuracy: 0.9633
Epoch 5/10
170/170 [==============================] - 53s 314ms/step - loss: 0.0642 - accuracy: 0.9858 - val_loss: 0.1486 - val_accuracy: 0.9660
Epoch 6/10
170/170 [==============================] - 52s 307ms/step - loss: 0.0482 - accuracy: 0.9898 - val_loss: 0.1512 - val_accuracy: 0.9670
Epoch 7/10
170/170 [==============================] - 53s 310ms/step - loss: 0.0378 - accuracy: 0.9919 - val_loss: 0.1488 - val_ac

INFO:tensorflow:Assets written to: embedding_300_dim\assets


# Change max Padding

Result do not improve increasing sentence length. So we choose lower length to decrease computational cost:

In [10]:
max_lens = [50, 300]
model_names = ['max_len_50', 'max_len_300']

for modelname, max_len in zip(model_names, max_lens):
    clear_session()
    
    codes = Codemaps(traindata, max_len, 5)
    
    lstm_units = 10
    embeddingW_dim, embeddingS_dim = 10, 10
    model = build_network(codes)
    
    Xt = codes.encode_words(traindata)
    Yt = codes.encode_labels(traindata)
    Xv = codes.encode_words(valdata)
    Yv = codes.encode_labels(valdata)
    
    model.fit(Xt, Yt, batch_size=32, epochs=10, validation_data=(Xv, Yv), verbose=1)
    model.save(modelname)
    codes.save(modelname)

Epoch 1/10
170/170 [==============================] - 25s 93ms/step - loss: 1.0057 - accuracy: 0.8717 - val_loss: 0.5611 - val_accuracy: 0.8783
Epoch 2/10
170/170 [==============================] - 15s 88ms/step - loss: 0.4809 - accuracy: 0.8844 - val_loss: 0.4486 - val_accuracy: 0.8787
Epoch 3/10
170/170 [==============================] - 14s 81ms/step - loss: 0.3519 - accuracy: 0.8865 - val_loss: 0.3384 - val_accuracy: 0.8940
Epoch 4/10
170/170 [==============================] - 14s 85ms/step - loss: 0.2631 - accuracy: 0.9212 - val_loss: 0.2843 - val_accuracy: 0.9238
Epoch 5/10
170/170 [==============================] - 14s 85ms/step - loss: 0.2237 - accuracy: 0.9364 - val_loss: 0.2604 - val_accuracy: 0.9286
Epoch 6/10
170/170 [==============================] - 14s 82ms/step - loss: 0.2002 - accuracy: 0.9391 - val_loss: 0.2478 - val_accuracy: 0.9301
Epoch 7/10
170/170 [==============================] - 14s 82ms/step - loss: 0.1809 - accuracy: 0.9420 - val_loss: 0.2319 - val_accuracy:

INFO:tensorflow:Assets written to: max_len_50\assets


Epoch 1/10
170/170 [==============================] - 87s 458ms/step - loss: 0.9869 - accuracy: 0.8720 - val_loss: 0.5608 - val_accuracy: 0.8781
Epoch 2/10
170/170 [==============================] - 71s 415ms/step - loss: 0.4790 - accuracy: 0.8835 - val_loss: 0.4366 - val_accuracy: 0.8786
Epoch 3/10
170/170 [==============================] - 72s 423ms/step - loss: 0.3409 - accuracy: 0.8901 - val_loss: 0.3217 - val_accuracy: 0.9078
Epoch 4/10
170/170 [==============================] - 64s 374ms/step - loss: 0.2539 - accuracy: 0.9295 - val_loss: 0.2733 - val_accuracy: 0.9284
Epoch 5/10
170/170 [==============================] - 70s 411ms/step - loss: 0.2182 - accuracy: 0.9374 - val_loss: 0.2533 - val_accuracy: 0.9300
Epoch 6/10
170/170 [==============================] - 70s 409ms/step - loss: 0.1975 - accuracy: 0.9390 - val_loss: 0.2376 - val_accuracy: 0.9316
Epoch 7/10
170/170 [==============================] - 61s 361ms/step - loss: 0.1771 - accuracy: 0.9419 - val_loss: 0.2202 - val_ac

INFO:tensorflow:Assets written to: max_len_300\assets


# Change sufix length

The suffix length that yields the best results for predicting the word category is 5 characters. Suffixes have variable lengths, and in each case, a different number of characters might be needed. However, the model cannot know this in advance, so a fixed length must be chosen. In this case, it is observed that on average, the length that works best is 5 characters, although the difference is small.

In [11]:
suf_lens = [3, 7]
model_names = ['suf_len_3', 'suf_len_7']


for modelname, suf_len in zip(model_names, suf_lens):
    clear_session()
    
    codes = Codemaps(traindata, 50, suf_len)
    lstm_units = 10
    embeddingW_dim, embeddingS_dim = 10, 10
    model = build_network(codes)
    
    Xt = codes.encode_words(traindata)
    Yt = codes.encode_labels(traindata)
    Xv = codes.encode_words(valdata)
    Yv = codes.encode_labels(valdata)
    
    model.fit(Xt, Yt, batch_size=32, epochs=10, validation_data=(Xv, Yv), verbose=1)
    model.save(modelname)
    codes.save(modelname)

Epoch 1/10
170/170 [==============================] - 18s 70ms/step - loss: 1.0593 - accuracy: 0.8676 - val_loss: 0.5633 - val_accuracy: 0.8783
Epoch 2/10
170/170 [==============================] - 11s 65ms/step - loss: 0.4653 - accuracy: 0.8846 - val_loss: 0.4168 - val_accuracy: 0.8801
Epoch 3/10
170/170 [==============================] - 11s 65ms/step - loss: 0.3216 - accuracy: 0.8990 - val_loss: 0.3083 - val_accuracy: 0.9185
Epoch 4/10
170/170 [==============================] - 12s 69ms/step - loss: 0.2438 - accuracy: 0.9346 - val_loss: 0.2612 - val_accuracy: 0.9307
Epoch 5/10
170/170 [==============================] - 11s 67ms/step - loss: 0.2050 - accuracy: 0.9386 - val_loss: 0.2319 - val_accuracy: 0.9334
Epoch 6/10
170/170 [==============================] - 11s 66ms/step - loss: 0.1761 - accuracy: 0.9454 - val_loss: 0.2033 - val_accuracy: 0.9481
Epoch 7/10
170/170 [==============================] - 12s 72ms/step - loss: 0.1463 - accuracy: 0.9604 - val_loss: 0.1786 - val_accuracy:

INFO:tensorflow:Assets written to: suf_len_3\assets


Epoch 1/10
170/170 [==============================] - 19s 73ms/step - loss: 1.0440 - accuracy: 0.8697 - val_loss: 0.5669 - val_accuracy: 0.8783
Epoch 2/10
170/170 [==============================] - 11s 66ms/step - loss: 0.4663 - accuracy: 0.8845 - val_loss: 0.4208 - val_accuracy: 0.8791
Epoch 3/10
170/170 [==============================] - 11s 67ms/step - loss: 0.3230 - accuracy: 0.8928 - val_loss: 0.3147 - val_accuracy: 0.9079
Epoch 4/10
170/170 [==============================] - 12s 69ms/step - loss: 0.2412 - accuracy: 0.9301 - val_loss: 0.2768 - val_accuracy: 0.9216
Epoch 5/10
170/170 [==============================] - 11s 66ms/step - loss: 0.2090 - accuracy: 0.9378 - val_loss: 0.2605 - val_accuracy: 0.9249
Epoch 6/10
170/170 [==============================] - 12s 69ms/step - loss: 0.1874 - accuracy: 0.9402 - val_loss: 0.2445 - val_accuracy: 0.9279
Epoch 7/10
170/170 [==============================] - 12s 69ms/step - loss: 0.1653 - accuracy: 0.9479 - val_loss: 0.2318 - val_accuracy:

INFO:tensorflow:Assets written to: suf_len_7\assets


# Increase LSTM units

When we use 100 LSTM units in the model, the accuracy with the training data improves significantly. However, the accuracy with the validation data does not improve as much, and the difference between the two increases. This is due to overfitting, as expected when increasing the complexity of the model. We see that with 20 LSTM units, the result is similar to that with 100 LSTM units, and the overfitting is lower. Additionally, the computational cost of training the model is significantly lower with fewer LSTM units.

When using 300 LSTM units, the accuracy with the validation data does not improve either, as expected.

In conclusion, we see that it is better to use a limited number (20 is enough) of LSTM units, as this avoids overfitting, achieves similar results with the validation data, and the computational cost is lower.

In [12]:
clear_session()

codes = Codemaps(traindata, 50, 5)

# 100 unidades
lstm_units = 100
embeddingW_dim, embeddingS_dim = 20, 20
model = build_network(codes)
    
Xt = codes.encode_words(traindata)
Yt = codes.encode_labels(traindata)
Xv = codes.encode_words(valdata)
Yv = codes.encode_labels(valdata)

modelname = 'more_lstm_units'
model.fit(Xt, Yt, batch_size=32, epochs=10, validation_data=(Xv, Yv), verbose=1)
model.save(modelname)
codes.save(modelname)

Epoch 1/10
170/170 [==============================] - 27s 126ms/step - loss: 0.6492 - accuracy: 0.8805 - val_loss: 0.3438 - val_accuracy: 0.9041
Epoch 2/10
170/170 [==============================] - 21s 126ms/step - loss: 0.2205 - accuracy: 0.9342 - val_loss: 0.2028 - val_accuracy: 0.9425
Epoch 3/10
170/170 [==============================] - 21s 122ms/step - loss: 0.1401 - accuracy: 0.9551 - val_loss: 0.1735 - val_accuracy: 0.9547
Epoch 4/10
170/170 [==============================] - 21s 126ms/step - loss: 0.1008 - accuracy: 0.9716 - val_loss: 0.1563 - val_accuracy: 0.9575
Epoch 5/10
170/170 [==============================] - 23s 138ms/step - loss: 0.0739 - accuracy: 0.9797 - val_loss: 0.1461 - val_accuracy: 0.9630
Epoch 6/10
170/170 [==============================] - 24s 144ms/step - loss: 0.0579 - accuracy: 0.9839 - val_loss: 0.1549 - val_accuracy: 0.9625
Epoch 7/10
170/170 [==============================] - 25s 145ms/step - loss: 0.0465 - accuracy: 0.9871 - val_loss: 0.1461 - val_ac

INFO:tensorflow:Assets written to: more_lstm_units\assets


In [13]:
clear_session()

codes = Codemaps(traindata, 50, 5)

# 300 unidades
lstm_units = 100
embeddingW_dim, embeddingS_dim = 20, 20
model = build_network(codes)
    
Xt = codes.encode_words(traindata)
Yt = codes.encode_labels(traindata)
Xv = codes.encode_words(valdata)
Yv = codes.encode_labels(valdata)

modelname = '300_lstm_units'
model.fit(Xt, Yt, batch_size=32, epochs=10, validation_data=(Xv, Yv), verbose=1)
model.save(modelname)
codes.save(modelname)

Epoch 1/10
170/170 [==============================] - 37s 169ms/step - loss: 0.6440 - accuracy: 0.8795 - val_loss: 0.3499 - val_accuracy: 0.8958
Epoch 2/10
170/170 [==============================] - 25s 149ms/step - loss: 0.2242 - accuracy: 0.9322 - val_loss: 0.2152 - val_accuracy: 0.9398
Epoch 3/10
170/170 [==============================] - 27s 160ms/step - loss: 0.1444 - accuracy: 0.9539 - val_loss: 0.1901 - val_accuracy: 0.9522
Epoch 4/10
170/170 [==============================] - 27s 158ms/step - loss: 0.1004 - accuracy: 0.9709 - val_loss: 0.1692 - val_accuracy: 0.9597
Epoch 5/10
170/170 [==============================] - 27s 160ms/step - loss: 0.0752 - accuracy: 0.9791 - val_loss: 0.1596 - val_accuracy: 0.9620
Epoch 6/10
170/170 [==============================] - 28s 164ms/step - loss: 0.0595 - accuracy: 0.9836 - val_loss: 0.1628 - val_accuracy: 0.9619
Epoch 7/10
170/170 [==============================] - 27s 158ms/step - loss: 0.0480 - accuracy: 0.9868 - val_loss: 0.1587 - val_ac

INFO:tensorflow:Assets written to: 300_lstm_units\assets


# Repeat the exercise using prefix instead of sufix

In [14]:
class Codemaps_pref:

    def __init__(self, data, maxlen=None, preflen=None):
        """
        Default constructor
        :param data:
        :param maxlen:
        :param preflen:
        """
        if isinstance(data, Dataset) and maxlen is not None and preflen is not None:
            self.__create_indexes(data, maxlen, preflen)

        elif type(data) == str and maxlen is None and preflen is None:
            self.__load(data)

        else:
            print('codemaps: Invalid or missing parameters in constructor')
            exit()

    def __create_indexes(self, data, maxlen, preflen):
        """
        Create indexes from the learning corpus
        Extracts all the words and labels in the given sentences and creates indexes to encode them as numbers
        :param data:
        :param maxlen:
        :param preflen:
        :return:
        """

        self.maxlen = maxlen
        self.preflen = preflen
        words = set([])
        lc_words = set([])
        prefs = set([])
        labels = set([])

        for s in data.sentences():
            for t in s:
                words.add(t['form'])
                prefs.add(t['lc_form'][:self.preflen])
                labels.add(t['tag'])

        self.word_index = {w: i + 2 for i, w in enumerate(list(words))}
        self.word_index['PAD'] = 0  # Padding
        self.word_index['UNK'] = 1  # Unknown words

        self.pref_index = {s: i + 2 for i, s in enumerate(list(prefs))}
        self.pref_index['PAD'] = 0  # Padding
        self.pref_index['UNK'] = 1  # Unknown prefixes

        self.label_index = {t: i + 1 for i, t in enumerate(list(labels))}
        self.label_index['PAD'] = 0  # Padding

    def __load(self, name):
        """
        Load the indexes
        :param name:
        :return:
        """
        self.maxlen = 0
        self.preflen = 0
        self.word_index = {}
        self.pref_index = {}
        self.label_index = {}

        with open(name + ".idx") as f:
            for line in f.readlines():
                (t, k, i) = line.split()
                if t == 'MAXLEN':
                    self.maxlen = int(k)
                elif t == 'PREFLEN':
                    self.preflen = int(k)
                elif t == 'WORD':
                    self.word_index[k] = int(i)
                elif t == 'PREF':
                    self.pref_index[k] = int(i)
                elif t == 'LABEL':
                    self.label_index[k] = int(i)

    def save(self, name):
        """
        Save the indexes
        :param name:
        :return:
        """
        with open(name + ".idx", "w") as f:
            print('MAXLEN', self.maxlen, "-", file=f)
            print('PREFLEN', self.preflen, "-", file=f)
            for key in self.label_index: print('LABEL', key, self.label_index[key], file=f)
            for key in self.word_index: print('WORD', key, self.word_index[key], file=f)
            for key in self.pref_index: print('PREF', key, self.pref_index[key], file=f)

    def encode_words(self, data):
        """
        Encodes X (text to indexes)
        :param data:
        :return:
        """
        # Encodes the words and adds padding
        Xw = [[self.word_index[w['form']] if w['form'] in self.word_index else self.word_index['UNK'] for w in s] for s
              in data.sentences()]
        Xw = pad_sequences(maxlen=self.maxlen, sequences=Xw, padding="post", value=self.word_index['PAD'])
        # Encodes the prefixes and adds padding
        Xs = [[self.pref_index[w['lc_form'][:self.preflen]] if w['lc_form'][:self.preflen] in self.pref_index else
               self.pref_index['UNK'] for w in s] for s in data.sentences()]
        Xs = pad_sequences(maxlen=self.maxlen, sequences=Xs, padding="post", value=self.pref_index['PAD'])
        # Returns the sentences
        return [Xw, Xs]

    def encode_labels(self, data):
        """
        Encodes labels and adds padding
        :param data:
        :return:
        """
        Y = [[self.label_index[w['tag']] for w in s] for s in data.sentences()]
        Y = pad_sequences(maxlen=self.maxlen, sequences=Y, padding="post", value=self.label_index["PAD"])
        return np.array(Y)

    def get_n_words(self):
        """
        Returns the number of word indexes
        :return:
        """
        return len(self.word_index)

    def get_n_prefs(self):
        """
        Returns the number of prefix indexes
        :return:
        """
        return len(self.pref_index)

    def get_n_labels(self):
        """
        Returns the number of labels
        :return:
        """
        return len(self.label_index)

    def word2idx(self, w):
        """
        Returns the index of a word
        :param w:
        :return:
        """
        return self.word_index[w]

    def pref2idx(self, s):
        """
        Returns the index of a prefix
        :param s:
        :return:
        """
        return self.pref_index[s]

    def label2idx(self, l):
        """
        Returns the index of a label
        :param l:
        :return:
        """
        return self.label_index[l]

    def idx2label(self, i):
        """
        Returns the label given an index
        :param i:
        :return:
        """
        for l in self.label_index:
            if self.label_index[l] == i:
                return l
        raise KeyError


In [15]:
def build_network_pref(codes):
    """
    Function that builds the neural network.
    :param codes:
    :return:
    """
    global lstm_units, embeddingW_dim, embeddingP_dim
    
    # sizes
    n_words = codes.get_n_words() # Number of words in the vocabulary
    n_prefs = codes.get_n_prefs() # Number of prefixes in the vocabulary
    n_labels = codes.get_n_labels() # Number of word classes
    max_len = codes.maxlen # Maximum length of each sentence, for padding

    inptW = Input(shape=(max_len,))  # Word embedding layer
    embW = Embedding(input_dim=n_words, output_dim=embeddingW_dim,
                     input_length=max_len, mask_zero=True)(inptW)

    inptP = Input(shape=(max_len,))  # Prefix embedding layer
    embP = Embedding(input_dim=n_prefs, output_dim=embeddingP_dim,
                     input_length=max_len, mask_zero=True)(inptP)

    dropW = Dropout(0.1)(embW)
    dropP = Dropout(0.1)(embP)
    drops = concatenate([dropW, dropP])

    # biLSTM
    bilstm = Bidirectional(LSTM(units=lstm_units, return_sequences=True,
                                recurrent_dropout=0.1))(drops)
    # Fully connected layer with softmax activation function to obtain the labels
    out = TimeDistributed(Dense(n_labels, activation="softmax"))(bilstm)

    # Build and compile the model
    model = Model([inptW, inptP], out)
    model.compile(optimizer="adam",
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])

    return model

The result is worse if we use prefixes. This makes sense, as the type or category of a word can be better predicted by looking at its ending (suffix). In fact, the accuracy with the training data improves, but not the accuracy with the validation data. This implies that there is greater overfitting.

In [16]:
clear_session()

# Directory with the files
traindir = 'data/train'
validationdir = 'data/devel'
modelname = 'models/model_1'

# Load the training and validation corpora
# ******************************************************************
# The following features are assigned to each word:
# - lc_form: word in lowercase
# - form: raw word
# - start: position of the first character in the sentence
# - end: position of the last character in the sentence
# - tag: word classification, according to B-I-O
# ******************************************************************
traindata = Dataset(traindir)
valdata = Dataset(validationdir)

# Create the indexes
# ******************************************************************
# When initializing the object of the Codemaps class, sets with index identifiers are created for each word, suffix, and class.
# It is used to encode sentences
# ******************************************************************
max_len = 150 # Maximum length for padding
suf_len = 5 # Last characters of each word to obtain suffix
codes = Codemaps_pref(traindata, max_len, suf_len)

# Build the network
# ******************************************************************
# Bidirectional LSTM model, with Embedding and Dropout layer
# ******************************************************************
# These are some model parameters (imported into the build_network function as global variables)
lstm_units = 20
embeddingW_dim = 50
embeddingP_dim = 50

model = build_network_pref(codes)
model.summary()

# Map the datasets
# ******************************************************************
# Encodes each sentence by assigning the corresponding index to each word or suffix, and applies padding
# Also encodes the classifications
# ******************************************************************
Xt = codes.encode_words(traindata)
Yt = codes.encode_labels(traindata)
Xv = codes.encode_words(valdata)
Yv = codes.encode_labels(valdata)

# Train the model
model.fit(Xt, Yt, batch_size=32, epochs=10, validation_data=(Xv, Yv), verbose=1)

# Save the model and the indexes
model.save(modelname)
codes.save(modelname)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 150)]                0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 150)]                0         []                            
                                                                                                  
 embedding (Embedding)       (None, 150, 50)              483800    ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, 150, 50)              258750    ['input_2[0][0]']             
                                                                                              

INFO:tensorflow:Assets written to: models/model_1\assets
